In [6]:
import requests
import json
from bs4 import BeautifulSoup
import time

# Extract

In [2]:
!pip install requests

def extract(rodada: int):
    """Função que recebe uma entrada "rodada" e retorna um dados json do site football-data.org

    Args:
        rodada (int): Rodada correspondente da premier league.

    Returns:
        dict: Dicionário que contêm dados coletados da API
    """
    url = f"https://api.football-data.org/v4/competitions/PL/matches?matchday={rodada}"
    api_key = "APIKEY" #coloque sua api do football-data.org no lugar do texto APIKEY
    headers = {"X-Auth-Token": api_key}

    response = requests.get(url, headers=headers)
    data = response.json()
    return data

# Transform

In [7]:
def transform(data : dict):
    """Função que recebe um dicionário simples sobre as partidas da premier league e 
    retorna um dicionário com mais informações como clima e temperatura em graus celsius no horário das partidas.

    Args:
        data (dict): Dicionário contendo horário e times das partidas.

    Returns:
        dict: Dicionário contendo dados da partida e clima no horário dos jogos.
    """
    partidas = {}
    data_partidas = data['matches']
    numero_partida = 0 # variavel apenas para nomear a chave do dicionario para ser unico
    
    for partida in data_partidas:
        time_mandante = partida['homeTeam']['name']
        time_visitante = partida['awayTeam']['name']
        utc_data = partida['utcDate']
        
        print(f"Mandante: {time_mandante}")
        print(f"Visitante: {time_visitante}")
        print(f"Data: {utc_data[:10]}")
        print(f"Horário: {utc_data[11:16]} UTC")
        print("-" * 20)
        partidas[numero_partida] = {'mandante': time_mandante, 'visitante': time_visitante, 'utc-data': utc_data}
        
        numero_partida += 1

    r = requests.get('https://en.wikipedia.org/wiki/List_of_Premier_League_clubs')

    data = r.text

    soup = BeautifulSoup(data, "html.parser")
    table = soup.find("table", {"class": "wikitable"})

    # Iterate through each row of the table (excluding the header row)
    for row in table.find_all("tr")[1:]:
        columns = row.find_all("td")
        if len(columns) >= 2:
            team_name_cell = columns[0]
            city_cell = columns[1]
            team_name = team_name_cell.get_text(strip=True)
            
            city_links = city_cell.find_all("a")
            if city_links:
                nome_cidade = city_links[0].get_text(strip=True)

            for key, value in partidas.items():
                if team_name in value['mandante']:
                    partidas[key]['cidade'] = nome_cidade

    api_key = 'APIKEY' # é necessário uma weatherbit api key, coloque sua chave aqui no lugar do texto "APIKEY"
    
    url = 'https://api.weatherbit.io/v2.0/forecast/hourly'

    # Parameters for the API request

    for key, value in partidas.items():
    #print(value['cidade'], value['utc-data'].replace('Z', ''))

        params = {'key': api_key,
            'city': value['cidade'],
            'country': 'GB',
            'start_time': value['utc-data'].replace('Z', ''),
            'end_time': value['utc-data'].replace('Z', ''),
            'tz': 'UTC'
        }

        # Make the API request
        response = requests.get(url, params=params)

        # Parse the response JSON
        data_clima = response.json()

        # Extrai dados do site weatherbit
        if 'data' in data_clima:
            visibilidade_nuvens = data_clima['data'][0]['clouds']
            precipitacao = data_clima['data'][0]['precip']
            temperatura = data_clima['data'][0]['temp']
            tempo = ''
            chuva = precipitacao > 0
            
            if visibilidade_nuvens < 20 and not chuva:
                tempo = 'Tempo ensolarado'
            elif visibilidade_nuvens < 70 and not chuva:
                tempo = "Tempo parcialmente nublado"
            elif chuva:
                tempo = "Tempo chuvoso"
            else:
                tempo = "Tempo nublado"
        else:
            print("Dados de clima indisponíveis.")
        partidas[key]['tempo'] = tempo
        partidas[key]['temperatura'] = f"{temperatura}°C"
        time.sleep(5)
    
    return partidas
    



# Load

In [4]:
def load(data : dict):
    """Procedimento no qual cria um arquivo json contendo os dados das partidas coletados anteriormente.

    Args:
        data (dict): Dicionário contendo dados da partida e clima no horário dos jogos.
    """
    arquivo = "Output/partidas_premier_league.json"
    # salva a data em um arquivo json
    with open(arquivo, "w") as json_file:
        json.dump(data, json_file)

# Main

In [8]:
raw_data = extract(rodada=3)
data = transform(raw_data)
# save the data in a json file
load(data)

Mandante: Chelsea FC
Visitante: Luton Town FC
Data: 2023-08-25
Horário: 19:00 UTC
--------------------
Mandante: AFC Bournemouth
Visitante: Tottenham Hotspur FC
Data: 2023-08-26
Horário: 11:30 UTC
--------------------
Mandante: Everton FC
Visitante: Wolverhampton Wanderers FC
Data: 2023-08-26
Horário: 14:00 UTC
--------------------
Mandante: Manchester United FC
Visitante: Nottingham Forest FC
Data: 2023-08-26
Horário: 14:00 UTC
--------------------
Mandante: Arsenal FC
Visitante: Fulham FC
Data: 2023-08-26
Horário: 14:00 UTC
--------------------
Mandante: Brentford FC
Visitante: Crystal Palace FC
Data: 2023-08-26
Horário: 14:00 UTC
--------------------
Mandante: Brighton & Hove Albion FC
Visitante: West Ham United FC
Data: 2023-08-26
Horário: 16:30 UTC
--------------------
Mandante: Sheffield United FC
Visitante: Manchester City FC
Data: 2023-08-27
Horário: 13:00 UTC
--------------------
Mandante: Burnley FC
Visitante: Aston Villa FC
Data: 2023-08-27
Horário: 13:00 UTC
---------------